In [1]:
import numpy as np 
import pandas as pd 

!ls docs/ 

all_paths_test.txt   data5.npy		 path_tse_train.txt
all_paths_train.txt  data_ecg_emg.npy	 seizures_types_v01.xlsx
data1.json	     data_ecg.npy	 seizures_v32r.xlsx
data3.csv	     get_eeg_windows.py  temp_csv.csv
data3.npy	     lse_list.npy


In [2]:
!pip install jupyterthemes

     |████████████████████████████████| 7.0MB 118kB/s eta 0:00:01
     |████████████████████████████████| 51kB 113kB/s ta 0:00:011


### Organizando tabela com informações

In [55]:
legends = pd.read_excel('docs/seizures_v32r.xlsx',sheet_name='legends')
test = pd.read_excel('docs/seizures_v32r.xlsx',sheet_name='dev_test')
train = pd.read_excel('docs/seizures_v32r.xlsx',sheet_name='train')
calibration = pd.read_excel('docs/seizures_v32r.xlsx',sheet_name='calibration')


In [109]:
col = ['file','freq','time','montage']

info_test = pd.DataFrame(data=np.load('info_test.npy'),columns=col)
info_train  = pd.DataFrame(data=np.load('info_train.npy'),columns=col)

### Organizando teste com crise

In [143]:
info_test.head()

,file,freq,time,montage
0,dev_test/01_tcp_ar/002/00000258/s002_2003_07_2...,400,20,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
1,dev_test/01_tcp_ar/002/00000258/s002_2003_07_2...,400,272,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
2,dev_test/01_tcp_ar/002/00000258/s003_2003_07_2...,400,234,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
3,dev_test/01_tcp_ar/002/00000258/s003_2003_07_2...,400,224,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
4,dev_test/01_tcp_ar/002/00000258/s003_2003_07_2...,400,203,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."


In [144]:
aux_test = test.iloc[1:1413,11:15].fillna(0)
aux_test = aux_test[aux_test['Seizure Time']!= 0]
aux_test.index = np.arange(len(aux_test))

for i in range(len(aux_test)):
    aux_test.iloc[i,0] = aux_test.iloc[i,0].replace('./edf/','')

print(aux_test.shape)
aux_test.head()


(685, 4)


,Filename,Seizure Time,Unnamed: 13,Seizure Type
0,dev_test/01_tcp_ar/002/00000258/s002_2003_07_2...,15.9375,136.4375,TCSZ
1,dev_test/01_tcp_ar/002/00000258/s003_2003_07_2...,13.0225,143.3225,TCSZ
2,dev_test/01_tcp_ar/002/00000258/s003_2003_07_2...,45.9300,162.2325,TCSZ
3,dev_test/01_tcp_ar/002/00000258/s003_2003_07_2...,8.8275,202.0000,TCSZ
4,dev_test/01_tcp_ar/002/00000258/s003_2003_07_2...,3.9625,115.0600,TCSZ


In [145]:
freq = []
mont = []
time = []

for i in range(len(info_test)):
    for ii in range(len(aux_test)):
        if str(info_test.iloc[i,0]).replace('.edf','') == str(aux_test.iloc[ii,0]).replace('.tse',''):
            freq.append(info_test.iloc[i,1])
            mont.append(info_test.iloc[i,-1])
            time.append(info_test.iloc[i,2])
            

In [147]:
aux_test['freq']=freq
aux_test['time']=time 
aux_test['montage']=mont

aux_test.columns = ['file','begin','end','type','freq','time','montage']
print(aux_test.shape)
aux_test.head()

(685, 7)


,file,begin,end,type,freq,time,montage
0,dev_test/01_tcp_ar/002/00000258/s002_2003_07_2...,15.9375,136.4375,TCSZ,400,272,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
1,dev_test/01_tcp_ar/002/00000258/s003_2003_07_2...,13.0225,143.3225,TCSZ,400,234,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
2,dev_test/01_tcp_ar/002/00000258/s003_2003_07_2...,45.9300,162.2325,TCSZ,400,224,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
3,dev_test/01_tcp_ar/002/00000258/s003_2003_07_2...,8.8275,202.0000,TCSZ,400,203,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
4,dev_test/01_tcp_ar/002/00000258/s003_2003_07_2...,3.9625,115.0600,TCSZ,400,205,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."


### Organizando traino com crise

In [148]:
info_train.head()

,file,freq,time,montage
0,train/01_tcp_ar/000/00000077/s003_2010_01_21/0...,250,1205,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
1,train/01_tcp_ar/002/00000254/s005_2010_11_15/0...,250,1442,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
2,train/01_tcp_ar/002/00000254/s006_2011_07_01/0...,250,1197,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
3,train/01_tcp_ar/002/00000254/s007_2013_03_25/0...,250,1244,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
4,train/01_tcp_ar/002/00000272/s007_2003_07_03/0...,400,194,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."


In [149]:
aux_train = train.iloc[1:6101,11:15].fillna(0)
aux_train = aux_train[aux_train['Seizure Time']!= 0]
aux_train.index = np.arange(len(aux_train))

for i in range(len(aux_train)):
    aux_train.iloc[i,0] = aux_train.iloc[i,0].replace('./edf/','')

print(aux_train.shape)
aux_train.head()


(2370, 4)


,Filename,Seizure Time,Unnamed: 13,Seizure Type
0,train/01_tcp_ar/004/00000492/s003_2003_07_18/0...,6.2825,35.8875,GNSZ
1,train/01_tcp_ar/004/00000492/s003_2003_07_18/0...,33.1425,53.0000,GNSZ
2,train/01_tcp_ar/004/00000492/s003_2003_07_18/0...,9.7925,29.1250,GNSZ
3,train/01_tcp_ar/004/00000492/s003_2003_07_18/0...,10.2775,35.7775,GNSZ
4,train/01_tcp_ar/004/00000492/s003_2003_07_18/0...,102.2525,142.9800,GNSZ


In [150]:
freq = []
mont = []
time = []

for i in range(len(info_train)):
    for ii in range(len(aux_train)):
        if str(info_train.iloc[i,0]).replace('.edf','') == str(aux_train.iloc[ii,0]).replace('.tse',''):
            freq.append(info_train.iloc[i,1])
            mont.append(info_train.iloc[i,-1])
            time.append(info_train.iloc[i,2])

In [151]:
aux_train['freq']=freq
aux_train['time']=time 
aux_train['montage']=mont

aux_train.columns = ['file','begin','end','type','freq','time','montage']
print(aux_train.shape)
aux_train.head()

(2370, 7)


,file,begin,end,type,freq,time,montage
0,train/01_tcp_ar/004/00000492/s003_2003_07_18/0...,6.2825,35.8875,GNSZ,400,165,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
1,train/01_tcp_ar/004/00000492/s003_2003_07_18/0...,33.1425,53.0000,GNSZ,400,184,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
2,train/01_tcp_ar/004/00000492/s003_2003_07_18/0...,9.7925,29.1250,GNSZ,400,129,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
3,train/01_tcp_ar/004/00000492/s003_2003_07_18/0...,10.2775,35.7775,GNSZ,400,339,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
4,train/01_tcp_ar/004/00000492/s003_2003_07_18/0...,102.2525,142.9800,GNSZ,400,339,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."


In [153]:
test_seizure = aux_test
train_seizure = aux_train

### Organizando teste  sem crise

In [154]:
info_test.head()

,file,freq,time,montage
0,dev_test/01_tcp_ar/002/00000258/s002_2003_07_2...,400,20,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
1,dev_test/01_tcp_ar/002/00000258/s002_2003_07_2...,400,272,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
2,dev_test/01_tcp_ar/002/00000258/s003_2003_07_2...,400,234,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
3,dev_test/01_tcp_ar/002/00000258/s003_2003_07_2...,400,224,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
4,dev_test/01_tcp_ar/002/00000258/s003_2003_07_2...,400,203,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."


In [155]:
aux_test = test.iloc[1:1413,11:15].fillna(0)
aux_test = aux_test[aux_test['Seizure Time']== 0]
aux_test.index = np.arange(len(aux_test))

for i in range(len(aux_test)):
    aux_test.iloc[i,0] = aux_test.iloc[i,0].replace('./edf/','')

print(aux_test.shape)
aux_test.head()

(727, 4)


,Filename,Seizure Time,Unnamed: 13,Seizure Type
0,dev_test/01_tcp_ar/002/00000258/s002_2003_07_2...,0.0,0.0,0
1,dev_test/01_tcp_ar/006/00000629/s003_2003_07_2...,0.0,0.0,0
2,dev_test/01_tcp_ar/006/00000629/s004_2003_07_2...,0.0,0.0,0
3,dev_test/01_tcp_ar/006/00000629/s004_2003_07_2...,0.0,0.0,0
4,dev_test/01_tcp_ar/006/00000629/s004_2003_07_2...,0.0,0.0,0


In [156]:
freq = []
mont = []
time = []

for i in range(len(info_test)):
    for ii in range(len(aux_test)):
        if str(info_test.iloc[i,0]).replace('.edf','') == str(aux_test.iloc[ii,0]).replace('.tse',''):
            freq.append(info_test.iloc[i,1])
            mont.append(info_test.iloc[i,-1])
            time.append(info_test.iloc[i,2])

In [158]:
aux_test['freq']=freq
aux_test['time']=time 
aux_test['montage']=mont
aux_test = aux_test.iloc[:,[0,4,5,6]]
aux_test.columns = ['file','freq','time','montage']
print(aux_test.shape)
aux_test.head()

(727, 4)


,file,freq,time,montage
0,dev_test/01_tcp_ar/002/00000258/s002_2003_07_2...,400,20,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
1,dev_test/01_tcp_ar/006/00000629/s003_2003_07_2...,400,185,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
2,dev_test/01_tcp_ar/006/00000629/s004_2003_07_2...,400,295,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
3,dev_test/01_tcp_ar/006/00000629/s004_2003_07_2...,400,369,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
4,dev_test/01_tcp_ar/006/00000629/s004_2003_07_2...,400,305,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."


In [159]:
test_n_seizure = aux_test

### Organizando treino sem crise

In [161]:
info_train.head()

,file,freq,time,montage
0,train/01_tcp_ar/000/00000077/s003_2010_01_21/0...,250,1205,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
1,train/01_tcp_ar/002/00000254/s005_2010_11_15/0...,250,1442,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
2,train/01_tcp_ar/002/00000254/s006_2011_07_01/0...,250,1197,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
3,train/01_tcp_ar/002/00000254/s007_2013_03_25/0...,250,1244,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
4,train/01_tcp_ar/002/00000272/s007_2003_07_03/0...,400,194,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."


In [167]:
aux_train = train.iloc[1:6101,11:15].fillna(0)
aux_train = aux_train[aux_train['Seizure Time']== 0]
aux_train.index = np.arange(len(aux_train))

for i in range(len(aux_train)):
    aux_train.iloc[i,0] = aux_train.iloc[i,0].replace('./edf/','')

print(aux_train.shape)
aux_train.head(10)

(3730, 4)


,Filename,Seizure Time,Unnamed: 13,Seizure Type
0,train/01_tcp_ar/000/00000077/s003_2010_01_21/0...,0.0,0.0,0
1,train/01_tcp_ar/002/00000254/s005_2010_11_15/0...,0.0,0.0,0
2,train/01_tcp_ar/002/00000254/s006_2011_07_01/0...,0.0,0.0,0
3,train/01_tcp_ar/002/00000254/s007_2013_03_25/0...,0.0,0.0,0
4,train/01_tcp_ar/002/00000272/s007_2003_07_03/0...,0.0,0.0,0
5,train/01_tcp_ar/002/00000272/s007_2003_07_03/0...,0.0,0.0,0
6,train/01_tcp_ar/002/00000272/s007_2003_07_03/0...,0.0,0.0,0
7,train/01_tcp_ar/002/00000272/s007_2003_07_03/0...,0.0,0.0,0
8,train/01_tcp_ar/002/00000272/s007_2003_07_03/0...,0.0,0.0,0
9,train/01_tcp_ar/002/00000272/s007_2003_07_03/0...,0.0,0.0,0


In [168]:
freq = []
mont = []
time = []

for i in range(len(info_train)):
    for ii in range(len(aux_train)):
        if str(info_train.iloc[i,0]).replace('.edf','') == str(aux_train.iloc[ii,0]).replace('.tse',''):
            freq.append(info_train.iloc[i,1])
            mont.append(info_train.iloc[i,-1])
            time.append(info_train.iloc[i,2])

In [171]:
aux_train['freq']=freq
aux_train['time']=time 
aux_train['montage']=mont

aux_train = aux_train.iloc[:,[0,4,5,6]]
aux_test.columns = ['file','freq','time','montage']

#aux_train.columns = ['file','begin','end','type','freq','time','montage']
print(aux_train.shape)
aux_train.head()

(3730, 4)


,file,freq,time,montage
0,train/01_tcp_ar/000/00000077/s003_2010_01_21/0...,250,1205,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
1,train/01_tcp_ar/002/00000254/s005_2010_11_15/0...,250,1442,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
2,train/01_tcp_ar/002/00000254/s006_2011_07_01/0...,250,1197,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
3,train/01_tcp_ar/002/00000254/s007_2013_03_25/0...,250,1244,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."
4,train/01_tcp_ar/002/00000272/s007_2003_07_03/0...,400,194,"[EEG FP1-REF, EEG FP2-REF, EEG F3-REF, EEG F4-..."


In [172]:
train_n_seizure = aux_train

In [173]:
np.save('train_n_seizure.npy',train_n_seizure.to_numpy())
np.save('test_n_seizure.npy',test_n_seizure.to_numpy())
np.save('test_seizure.npy',test_seizure.to_numpy())
np.save('train_seizure.npy',train_seizure.to_numpy())